## Import libraries and customize matplotlib styles params

In [59]:
import matplotlib.pyplot as plt
import numpy as np
import sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
   from google.colab import output
   output.enable_custom_widget_manager()
   !pip install --quiet plotly
   
import plotly.express as px


## Main Functions

In [60]:
def map_fun(a,b,n=10,mode='log',Npoints=100,decrease_diff=False):
    """ xn[0,1] --> xm[a,b]
        decrease_diff     not decrease_diff  
        ################################   
        1-(x-1)**2  even  |   x**2,x**3
        1+(x-1)**3  odd   |                 
                          |
        log10(9*x+1)      |  (10**x-1)/9      """
        
    
    xn= np.linspace(0,1,Npoints)
    
    if mode == 'log':
       if decrease_diff: xs=np.emath.logn(n, (n-1)*xn+1)   
       else:             xs=(n**(xn)-1)/(n-1)  
    if mode == 'exp':
        if decrease_diff:  xs = 1 -(-1)**n*(xn-1)**n
        else:              xs = xn**n

    xm = (b-a)*xs + a

    return xm,xn

def mandelbrot_set3D(Nmax=500,Msize=100,lims={ "xlims":[-1.9,0.7],"ylims":[-1.2,1.2]}):
    
    dx = np.diff(lims["xlims"])[0]
    dy = np.diff(lims["ylims"])[0]
    if   dx>dy: Nx,Ny = round(dx/dy * Msize),round(Msize)
    elif dy>dx: Ny,Nx = round(dy/dx * Msize),round(Msize)
    else:       Ny,Nx = round(Msize),round(Msize)
    
    x,_ = map_fun(    lims["xlims"][0],lims["xlims"][1],n=3,mode='exp',Npoints=Nx,decrease_diff=False)
    y   = np.linspace(lims["ylims"][0],lims["ylims"][1],Ny) 
    CRe,CIm = np.meshgrid(x,y)
    C       = CRe + CIm*1j

    def trunc(values, decs=6):
        return np.trunc(values*10**decs)/(10**decs)
     
    Z0    = np.zeros(Nmax, dtype=np.complex_)
    Z0[0] = 0

    def iterations(Nmax,C,Z=Z0):
    
        for i in range(Nmax-1):
            Z[i+1] = Z[i]**2 + C
            if np.abs(Z[i])>=2:
               break
        if i==Nmax-2: return {'IN_Mandelbrot_set':True, 'data':Z     }
        else:         return {'IN_Mandelbrot_set':False,'data':np.nan}

    Nunq    = 10
    periods = np.array([])
    Cre     = np.array([])   
    Cim     = np.array([]) 
    Zre     = np.array([])

    for c in C.flatten():
            
            orbit = iterations(Nmax,c)

            if orbit['IN_Mandelbrot_set']:

               Z       = orbit['data']
               zre     = np.real(Z[-Nunq:])
               zre     = np.unique( trunc(zre) )
               Zre     = np.concatenate( (Zre,zre) )
               nper    = zre.size
               Cre     = np.concatenate( (Cre, np.real(c)*np.ones(nper)) )
               Cim     = np.concatenate( (Cim, np.imag(c)*np.ones(nper)) )
               periods = np.concatenate( (periods, nper*np.ones(nper)  ) )

    ind = np.argsort(periods)

    return Cre[ind],Cim[ind],Zre[ind],periods[ind]


## Calc Data

In [65]:
Cre,Cim,Zre,periods= mandelbrot_set3D(Nmax=1000,Msize=201,lims={ "xlims":[-1.9,0.5],"ylims":[-1,1]})

## Plot

In [66]:

data = {'Re[C]': Cre, 'Im[C]':Cim,'Re[Z]':Zre,'Periods':[str(p) for p in periods]}

fig = px.scatter_3d(data, x="Re[C]", y="Im[C]", z="Re[Z]", color='Periods')

fig.update_traces(marker=dict(size=2, opacity=1),selector=dict(mode='markers'))

fig.update_layout(
    # width  = 700,
    # height = 700,
    title  = "Mandelbrot Set 3D",
    template="plotly_dark",
    font={'size':14},
)
fig.write_html("imgs/Mandelbrot_3Dplot.html")
fig.show()


In [63]:
np.argwhere(np.linspace(-1,1,501)==0)

array([[250]], dtype=int64)